In [1]:
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
import sklearn
from pandas import Series, DataFrame
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
df = pd.read_csv('E:/HDD Documents/HDD Documents/School/WGU/d208_predictive_modeling/medical/medical_clean.csv')

In [3]:
df.head()

,CaseOrder,Customer_id,Interaction,UID,City,State,County,Zip,Lat,Lng,...,TotalCharge,Additional_charges,Item1,Item2,Item3,Item4,Item5,Item6,Item7,Item8
0,1,C412403,8cd49b13-f45a-4b47-a2bd-173ffa932c2f,3a83ddb66e2ae73798bdf1d705dc0932,Eva,AL,Morgan,35621,34.34960,-86.72508,...,3726.702860,17939.403420,3,3,2,2,4,3,3,4
1,2,Z919181,d2450b70-0337-4406-bdbb-bc1037f1734c,176354c5eef714957d486009feabf195,Marianna,FL,Jackson,32446,30.84513,-85.22907,...,4193.190458,17612.998120,3,4,3,4,4,4,3,3
2,3,F995323,a2057123-abf5-4a2c-abad-8ffe33512562,e19a0fa00aeda885b8a436757e889bc9,Sioux Falls,SD,Minnehaha,57110,43.54321,-96.63772,...,2434.234222,17505.192460,2,4,4,4,3,4,3,3
3,4,A879973,1dec528d-eb34-4079-adce-0d7a40e82205,cd17d7b6d152cb6f23957346d11c3f07,New Richland,MN,Waseca,56072,43.89744,-93.51479,...,2127.830423,12993.437350,3,5,5,3,4,5,5,5
4,5,C544523,5885f56b-d6da-43a3-8760-83583af94266,d2f0425877b10ed6bb381f3e2579424a,West Point,VA,King William,23181,37.59894,-76.88958,...,2113.073274,3716.525786,2,1,3,3,5,3,4,3


In [4]:
df.isna().any()

CaseOrder             False
Customer_id           False
Interaction           False
UID                   False
City                  False
State                 False
County                False
Zip                   False
Lat                   False
Lng                   False
Population            False
Area                  False
TimeZone              False
Job                   False
Children              False
Age                   False
Income                False
Marital               False
Gender                False
ReAdmis               False
VitD_levels           False
Doc_visits            False
Full_meals_eaten      False
vitD_supp             False
Soft_drink            False
Initial_admin         False
HighBlood             False
Stroke                False
Complication_risk     False
Overweight            False
Arthritis             False
Diabetes              False
Hyperlipidemia        False
BackPain              False
Anxiety               False
Allergic_rhinitis   

In [5]:
df[df.duplicated()]

,CaseOrder,Customer_id,Interaction,UID,City,State,County,Zip,Lat,Lng,...,TotalCharge,Additional_charges,Item1,Item2,Item3,Item4,Item5,Item6,Item7,Item8


In [6]:
df.columns.duplicated().any()

False

In [8]:
df.drop(['Item8','Item7','Item6','Item5','Item4','Item3','Item2','Item1'],axis = 1,inplace=True)
#Drop demographic data
df.drop(['City','State','County','Zip','Lat','Lng','Population','TimeZone'],axis = 1,inplace=True)
#Drop interaction data not needed
df.drop(['Additional_charges','TotalCharge','Services','Interaction','UID'],axis = 1,inplace=True)
#Drop other data not needed
df.drop(['Customer_id','Job',],axis = 1, inplace=True)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 27 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   CaseOrder           10000 non-null  int64  
 1   Area                10000 non-null  object 
 2   Children            10000 non-null  int64  
 3   Age                 10000 non-null  int64  
 4   Income              10000 non-null  float64
 5   Marital             10000 non-null  object 
 6   Gender              10000 non-null  object 
 7   ReAdmis             10000 non-null  object 
 8   VitD_levels         10000 non-null  float64
 9   Doc_visits          10000 non-null  int64  
 10  Full_meals_eaten    10000 non-null  int64  
 11  vitD_supp           10000 non-null  int64  
 12  Soft_drink          10000 non-null  object 
 13  Initial_admin       10000 non-null  object 
 14  HighBlood           10000 non-null  object 
 15  Stroke              10000 non-null  object 
 16  Compl

In [10]:
target = 'ReAdmis'
for idx, c in enumerate(df.loc[:, df.columns == target]):
    if df.dtypes[c] == "object":
        print('\n{}. {} is categorical: {}.'.format(idx+1,c,df[c].unique()))
        for idx,name in enumerate(df[c].value_counts().index.tolist()):
            print('\t{:<8}:{:>6}'.format(name,df[c].value_counts()[idx]))
    else:
        print('\n{}. {} is numerical.'.format(idx+1, c))


1. ReAdmis is categorical: ['No' 'Yes'].
	No      :  6331
	Yes     :  3669


In [11]:
df['Marital']=np.where(df['Marital'] =='Widowed', 'Not_Married',df['Marital'])
df['Marital']=np.where(df['Marital'] =='Separated', 'Not_Married',df['Marital'])
df['Marital']=np.where(df['Marital'] =='Never Married', 'Not_Married',df['Marital'])
df['Marital']=np.where(df['Marital'] =='Divorced', 'Not_Married',df['Marital'])

In [12]:
df['Marital'].unique()

array(['Not_Married', 'Married'], dtype=object)

In [13]:
for idx, c in enumerate(df.loc[:, df.columns != target]):
    if df.dtypes[c] == "object":
        print('\n{}. {} is categorical: {}.'.format(idx+1,c,df[c].unique()))
        for idx,name in enumerate(df[c].value_counts().index.tolist()):
            print('\t{:<20}:{:>6}'.format(name,df[c].value_counts()[idx]))
            print('{}'.format(df[c].describe()))
    else:
        print('\n{}. {} is numerical.'.format(idx+1, c))


1. CaseOrder is numerical.

2. Area is categorical: ['Suburban' 'Urban' 'Rural'].
	Rural               :  3369
count     10000
unique        3
top       Rural
freq       3369
Name: Area, dtype: object
	Suburban            :  3328
count     10000
unique        3
top       Rural
freq       3369
Name: Area, dtype: object
	Urban               :  3303
count     10000
unique        3
top       Rural
freq       3369
Name: Area, dtype: object

3. Children is numerical.

4. Age is numerical.

5. Income is numerical.

6. Marital is categorical: ['Not_Married' 'Married'].
	Not_Married         :  7977
count           10000
unique              2
top       Not_Married
freq             7977
Name: Marital, dtype: object
	Married             :  2023
count           10000
unique              2
top       Not_Married
freq             7977
Name: Marital, dtype: object

7. Gender is categorical: ['Male' 'Female' 'Nonbinary'].
	Female              :  5018
count      10000
unique         3
top       Female
f

In [14]:
CategoricalData = df.select_dtypes(include = "object").columns
print(CategoricalData)

Index(['Area', 'Marital', 'Gender', 'ReAdmis', 'Soft_drink', 'Initial_admin',
       'HighBlood', 'Stroke', 'Complication_risk', 'Overweight', 'Arthritis',
       'Diabetes', 'Hyperlipidemia', 'BackPain', 'Anxiety',
       'Allergic_rhinitis', 'Reflux_esophagitis', 'Asthma'],
      dtype='object')


In [15]:
NumericalData = df.select_dtypes(include = "number").columns
print(NumericalData)

Index(['CaseOrder', 'Children', 'Age', 'Income', 'VitD_levels', 'Doc_visits',
       'Full_meals_eaten', 'vitD_supp', 'Initial_days'],
      dtype='object')
